In [1]:
## importazione dei dati ------------------------------------------------------------------------------------------------------------

from pandas import DataFrame
from sklearn.preprocessing import MinMaxScaler
import statsmodels.api as sm
import pandas as pd
import numpy as np
import statsmodels.api as sm
from scipy.linalg import toeplitz
from sklearn.preprocessing import PolynomialFeatures
from sklearn import linear_model
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_validate

df = pd.read_csv (r'/Users/patrickpinta/Desktop/1datasetpanda/soybean.csv',header=0,error_bad_lines=False) 

#costruzione del modello dei dati training set------------------------------------------------------------------------------------------------------------

xExpUsa= df.iloc[0:334,1:2]
xExpUsa = np.array(xExpUsa)
xExpBra=df.iloc[0:334,3:4]
xExpBra = np.array(xExpBra)
xExpArg=df.iloc[0:334,5:6]
xExpArg = np.array(xExpArg)
xImportChina=xExpUsa+xExpBra+xExpArg

polynomial_features= PolynomialFeatures(degree=2)
xImportChina = polynomial_features.fit_transform(xImportChina)

xBimestri=df.iloc[0:334,17:22] 
xBimestri = np.array(xBimestri)

xUsa=df.iloc[0:334,0:1]
xUsa = np.array(xUsa)
xBimInfExpUsa=xBimestri*xUsa

xBra=df.iloc[0:334,1:2]
xBra = np.array(xBra)
xBimInfExpBra=xBimestri*xBra

xArg=df.iloc[0:334,2:3]
xArg = np.array(xArg)
xBimInfExpArg=xBimestri*xArg


x=df.iloc[0:334,0:1]
x2=df.iloc[0:334,2:3]
x3=df.iloc[0:334,4:5]
x4=df.iloc[0:334,6:7]
x5=df.iloc[0:334,8:15]
x6=df.iloc[0:334,15:16]
x7=df.iloc[0:334,16:17]
x8=df.iloc[0:334,17:22]

x6=polynomial_features.fit_transform(x6)
x6=x6.tolist()
x6 = DataFrame (x6,columns=['sW0','sW1','sW2'])

x7=polynomial_features.fit_transform(x7)
x7=x7.tolist()
x7 = DataFrame (x7,columns=['sU0','sU1','sU2'])

xImportChina=xImportChina.tolist()
xImportChina = DataFrame (xImportChina,columns=['ImportChina','ImportChina1','ImportChina2'])
xBimInfExpUsa=xBimInfExpUsa.tolist()
xBimInfExpUsa = DataFrame (xBimInfExpUsa,columns=['bim1*expUsa','bim2*expUsa','bim3*expUsa','bim4*expUsa','bim5*expUsa'])
xBimInfExpBra=xBimInfExpBra.tolist()
xBimInfExpBra = DataFrame (xBimInfExpBra,columns=['bim1*expBra','bim2*expBra','bim3*expBra','bim4*expBra','bim5*expBra'])
xBimInfExpArg=xBimInfExpArg.tolist()
xBimInfExpArg = DataFrame (xBimInfExpArg,columns=['bim1*expArg','bim2*expArg','bim3*expArg','bim4*expArg','bim5*expArg'])
x=x.join(xImportChina.iloc[:,1:3])
x=x.join(x2)
x=x.join(x3)
x=x.join(x4)
x=x.join(x5)
x=x.join(x6.iloc[:,1:3])
x=x.join(x7.iloc[:,1:3])
x=x.join(xBimInfExpUsa)
x=x.join(xBimInfExpBra)
x=x.join(xBimInfExpArg)
x=x.join(x8)


y= df.iloc[0:334,23:24]


#dff=x.join(y)
#print(dff.corr(method='pearson'))


#stardadizzazione dei dati training set------------------------------------------------------------------------------------------------------------

scaler = MinMaxScaler(feature_range=(-1,1))

x_train_stand=(scaler.fit_transform(x))

#modello dati training set------------------------------------------------------------------------------------------------------------    
    
x_train_stand, y = np.array(x_train_stand), np.array(y)
x_train_stand = sm.add_constant(x_train_stand)

x = np.array(x)
x = sm.add_constant(x)
#procedura per ols model ------------------------------------------------------------------------------------------------------------

ols_model = sm.OLS(y, x_train_stand)
ols_results = ols_model.fit(cov_type='HC1') 
#print(ols_results.summary())

reg = linear_model.Ridge(alpha='0.2')
#reg = linear_model.LinearRegression()
reg= reg.fit(x_train_stand,y)

#forecast ------------------------------------------------------------------------------------------------------------
ypredTrain = reg.predict(x_train_stand)

print('r2 train: ',r2_score(y,ypredTrain))
print('mse train: ',mean_squared_error(y,ypredTrain))

scores=cross_validate(reg, x_train_stand, y, cv=5,scoring=('r2', 'neg_mean_squared_error'),return_train_score=True)
print('r2 cv: ', scores['test_r2'].mean())
print('mse cv: ', abs(scores['test_neg_mean_squared_error'].mean()))

#modellazione dei dati test set------------------------------------------------------------------------------------------------------------

xtestExpUsa= df.iloc[0:417,1:2]
xtestExpUsa = np.array(xtestExpUsa)
xtestExpBra=df.iloc[0:417,3:4]
xtestExpBra = np.array(xtestExpBra)
xtestExpArg=df.iloc[0:417,5:6]
xtestExpArg = np.array(xtestExpArg)
xtestImportChina=xtestExpUsa+xtestExpBra+xtestExpArg

polynomial_features= PolynomialFeatures(degree=2)
xtestImportChina = polynomial_features.fit_transform(xtestImportChina)

xtestBimestri=df.iloc[0:417,17:22] 
xtestBimestri = np.array(xtestBimestri)

xtestUsa=df.iloc[0:417,0:1]
xtestUsa = np.array(xtestUsa)
xtestBimInfExpUsa=xtestBimestri*xtestUsa

xtestBra=df.iloc[0:417,1:2]
xtestBra = np.array(xtestBra)
xtestBimInfExpBra=xtestBimestri*xtestBra

xtestArg=df.iloc[0:417,2:3]
xtestArg = np.array(xtestArg)
xtestBimInfExpArg=xtestBimestri*xtestArg


xtest=df.iloc[334:417,0:1]
x2test=df.iloc[334:417,2:3]
x3test=df.iloc[334:417,4:5]
x4test=df.iloc[334:417,6:7]
x5test=df.iloc[334:417,8:15]
x6test=df.iloc[0:417,15:16]
x7test=df.iloc[0:417,16:17]
x8test=df.iloc[334:417,17:22]

x6test=polynomial_features.fit_transform(x6test)
x6test=x6test.tolist()
x6test = DataFrame (x6test,columns=['sW0','sW1','sW2'])

x7test=polynomial_features.fit_transform(x7test)
x7test=x7test.tolist()
x7test = DataFrame (x7test,columns=['sU0','sU1','sU2'])

xtestImportChina=xtestImportChina.tolist()
xtestImportChina = DataFrame (xtestImportChina,columns=['ImportChina','ImportChina1','ImportChina2'])
xtestBimInfExpUsa=xtestBimInfExpUsa.tolist()
xtestBimInfExpUsa = DataFrame (xtestBimInfExpUsa,columns=['bim1*expUsa','bim2*expUsa','bim3*expUsa','bim4*expUsa','bim5*expUsa'])
xtestBimInfExpBra=xtestBimInfExpBra.tolist()
xtestBimInfExpBra = DataFrame (xtestBimInfExpBra,columns=['bim1*expBra','bim2*expBra','bim3*expBra','bim4*expBra','bim5*expBra'])
xtestBimInfExpArg=xtestBimInfExpArg.tolist()
xtestBimInfExpArg = DataFrame (xtestBimInfExpArg,columns=['bim1*expArg','bim2*expArg','bim3*expArg','bim4*expArg','bim5*expArg'])
xtest=xtest.join(xtestImportChina.iloc[334:417,1:3])
xtest=xtest.join(x2test)
xtest=xtest.join(x3test)
xtest=xtest.join(x4test)
xtest=xtest.join(x5test)
xtest=xtest.join(x6test.iloc[334:417,1:3])
xtest=xtest.join(x7test.iloc[334:417,1:3])
xtest=xtest.join(xtestBimInfExpUsa.iloc[334:417,0:5])
xtest=xtest.join(xtestBimInfExpBra.iloc[334:417,0:5])
xtest=xtest.join(xtestBimInfExpArg.iloc[334:417,0:5])
xtest=xtest.join(x8test)

ytest= df.iloc[334:417,23:24]

#stardadizzazione dei dati test set------------------------------------------------------------------------------------------------------------

scaler = MinMaxScaler(feature_range=(-1,1))

x_test_stand=(scaler.fit_transform(xtest))

#modello dati test set------------------------------------------------------------------------------------------------------------    

x_test_stand, ytest = np.array(x_test_stand), np.array(ytest)
x_test_stand = sm.add_constant(x_test_stand)

ypredTest = reg.predict(x_test_stand)

print('r2 test: ',r2_score(ytest,ypredTest))
print('mse test: ',mean_squared_error(ytest,ypredTest))

i=0
for i in range(83):
    print(ytest[i],ypredTest[i])

i=0
for i in range(37):
    print(xtest.iloc[7:8,i:(i+1)])


#i=0
#for i in range(38):
    #print(reg.coef_[0,i])
    



r2 train:  0.8632888129335632
mse train:  4985.805323194679
r2 cv:  0.8219349997559966
mse cv:  6434.271138035316
r2 test:  0.7377055956095088
mse test:  8563.508530876725
[863.25] [855.53479395]
[1032.88] [1093.69428248]
[1193.62] [1131.61759523]
[939.12] [958.17504778]
[1039.63] [938.78817311]
[1177.75] [1079.26492188]
[1038.63] [950.13670896]
[1001.62] [958.83479492]
[1001.12] [1035.25296814]
[1271.75] [1401.14553592]
[896.5] [656.68653604]
[1085.88] [1119.17598038]
[1425.5] [1406.65305813]
[878.38] [934.12901681]
[1264.4] [1103.28859499]
[895.38] [901.70430292]
[1052.] [1087.57228827]
[962.88] [944.41731828]
[895.38] [896.47161266]
[1491.13] [1245.75792697]
[999.63] [1028.69059426]
[1283.25] [1274.19118845]
[987.38] [1073.08434821]
[900.25] [851.14658573]
[948.38] [954.44088255]
[858.13] [896.89805839]
[880.75] [976.2556465]
[1052.25] [863.51780764]
[1350.] [1345.83682194]
[989.88] [905.60287381]
[1113.88] [1055.34298039]
[880.75] [863.91634976]
[1131.75] [1136.52306975]
[978.5] [9